In [1]:
#libraries needed for extraction and for formatting spreadsheet

import os
import Bio
from Bio import SeqIO
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from Bio.SeqRecord import SeqRecord
import xml.etree.ElementTree as ET
from openpyxl import Workbook
from openpyxl.styles import Color, PatternFill, Style


In [70]:
#Blasts all ORFS in './VGDB/pSS17' against the genomes.
#Genomes have the simplified name S## where ## are integers
#Creates a tempororary infile for the query, then deletes it.
#Switching to allow more processing in the future of the in-sequence (format changes)
#Output - xml files (outfmt=5) to './VG_BLAST_Hits/'
#folders must be made in advance if not already existing.

q = './VGDB/tempfile.fasta'
d = './BLAST_DB/DavisDB.fasta'
alreadydone = []

record = SeqIO.read('./VGDB/pSS17.gb','genbank')
for feat in record.features:
    if str(feat.location)!='[0:37447](+)' and feat.type=='gene':
        newid = feat.qualifiers['locus_tag'][0]
        start = int((str(feat.location).split('[')[1].split(':')[0]))
        end = int((str(feat.location).split(']')[0].split(':')[1]))
        sequence = str(record.seq[start:end])
    newseq = Seq(sequence)
    newseq_r = SeqRecord(newseq)
    newseq_r.id = newid
    newseq_r.name = newid
    if newid in alreadydone:
        continue
    alreadydone.append(newid)
    print(newseq_r)
    otf = open(q,'w')
    SeqIO.write(newseq_r, q, 'fasta')
    otf.close
    o = './pSS17_BLAST_HITS/'+newid+'.xml'
    os.system(str(NcbiblastnCommandline(query = q, db = d, out = o, outfmt=5)))
    os.remove(q)

ID: SS17_RS29095
Name: SS17_RS29095
Description: <unknown description>
Number of features: 0
Seq('ATGAAGAAAAAAATCATATTAGTGGTTTTTTTGTTGCTTATTCTCGTTGCTTGT...TAA', Alphabet())
ID: SS17_RS29020
Name: SS17_RS29020
Description: <unknown description>
Number of features: 0
Seq('ATGATGTTGCCAGTAGAGGTGAATACGCGAGAGGGGGAGTTAATATTTCAATCA...TGA', Alphabet())
ID: SS17_RS29025
Name: SS17_RS29025
Description: <unknown description>
Number of features: 0
Seq('ATGGCTACTTTTGATTTTACACACCTCAATGGATTAACACAAATCAAAGCCTTG...TAA', Alphabet())
ID: SS17_RS29030
Name: SS17_RS29030
Description: <unknown description>
Number of features: 0
Seq('GTGAATAAGAGTCGAAATAATGATGTAAGTGATAATTTAAATGCAGGAAATTTT...TAA', Alphabet())
ID: SS17_RS29035
Name: SS17_RS29035
Description: <unknown description>
Number of features: 0
Seq('ATGAATATAATATCTACGTTTTTAAGTACTGTTACAACACTAGTTGAATCAGGG...TAA', Alphabet())
ID: SS17_RS29040
Name: SS17_RS29040
Description: <unknown description>
Number of features: 0
Seq('ATGAAAAAAGTTGTTCCAATGGCATTTGTATCCGTTC

In [3]:
#Extract BLAST hits

#hitdict is a dictionary of all blast hits.
#it stores a list of important data for each hit for calculation.
#hits are checked to be one per contig at end.
#hitdict list order:
#query start, query stop, query length, alignment length,
#number of identical positions, number of 'positive' positions
#number of gaps.
#[qstart,qstop,qlen,alen,ids,pos,gaps]

hitdict = {}

#retrieves xml file, parses with element tree, root = top node
for file in os.listdir('./pSS17_BLAST_HITS'):
    if file.endswith('.xml'):
        queryg = os.path.splitext(file)[0]
        queryf = os.path.join('./pSS17_BLAST_HITS/'+file)
        tree = ET.parse(queryf)
        root = tree.getroot()

#Iteration - query information
#hit- hits within genome
#hsp- alignment infor within a hit
        for iteration in root.findall(".//Iteration"):
            qlen = iteration.find('Iteration_query-len').text
        for hit in root.findall(".//Hit"):
            contig = hit.find('Hit_id').text
            for hsp in hit.findall('.//Hit_hsps/Hsp'):
                hseq = hsp.find('Hsp_hseq').text
                qstart = hsp.find('Hsp_query-from').text
                qstop = hsp.find('Hsp_query-to').text
                hstart = hsp.find('Hsp_hit-from').text
                hstop = hsp.find('Hsp_hit-to').text
                alen = hsp.find('Hsp_align-len').text
                ids = hsp.find('Hsp_identity').text
                pos = hsp.find('Hsp_positive').text
                gaps = hsp.find('Hsp_gaps').text
                
            key = contig+'|'+queryg
            keylist = [item for item in hitdict.keys()]
            if key in keylist:
                print('overwrite at '+key)
                
            hitdict[key]=[qstart,qstop,qlen,alen,ids,pos,gaps]
     

#REORDERED GENOME LIST TO MATCH PHYLOGENETIC TREE IN FIGURE

#Create a table for in-text presentation
#Must run the hitdict creator first (two cells up)

wb = Workbook()
ws1 = wb.create_sheet(0)
ws1.title = 'In Text Figure'

fillyes = PatternFill(fill_type='solid', start_color='0000FF', end_color='0000FF')
fillno = PatternFill(fill_type='solid', start_color='CCCCCC', end_color='CCCCCC')


#Retrieve keys from hitdict for creating lists of genes and 
keys = hitdict.keys()

hn=[]
gn=[]
for item in keys:
    hn.append(item.split('|')[0].split('(')[0])
    gn.append(item.split('|')[1])
    
#Create sorted sets of the genes here.
gn = sorted(set(gn))
hn = sorted(set(hn))

#Exclude lists: lists of things that should be excluded from the table
#gnx - genes to exclude
#hnx - genomes (hits) to exclude

#S54 - Low quality.
#S17 - Returns very fragmented blast hits. Unknown reason - other quality metrics look fine

gnx = ['pSS17','EDL933_etpoperonpartialhlyC','Sakai_hlyA', 'Sakai_hlyB', 'Sakai_hlyC', 'Sakai_hlyD', 'pO157_ecf1', 'pO157_ecf1to4partial4', 'pO157_ecf2']

#Some are controls, salmonella, or duplicate samples
hnx = ['S66','S69','S70','S35','S36','S38','S40','S42','S44','S46','S49','S50','S53','S54','S63','S68','S71','S72','S88']

for item in hnx:
    if item in hn:
        hn.remove(item)

hnorder=['S83','S82','S84','S73','S61','S57','S60','S58','S80','S79','S52','S59','S65',
         'S43','S48','S64','S62','S47','S51','S56','S55','S45','S76','S75','S39','S77',
         'S81','S41','S78','S87','S67','S16','S15','S13','S20','S05','S74','S27','S17',
         'S22','S06','S85','S86','S14','S12','S31','S01','S34','S37','S07','S23','S32',
         'S02','S21','S10','S26','S18','S30','S24','S25','S29','S03','S19','S28','S11',
         'S04','S08','S33','S09']
if sorted(set(hn)) == sorted(set(hnorder)):
    print('set check OK')
    hn = hnorder
for item in hn:
    if item not in hnorder:
        print(item)

print(gn)
print(hn)

#Label columns and rows
for n in range(len(hn)):
    ws1.cell(row = n+2, column = 1).value = hn[n]
for n in range(len(gn)):
    ws1.cell(row = 1, column = n+2).value = gn[n]

#hitdict[key]=[qstart,qstop,qlen,alen,ids,pos,gaps]    
tempgn = []
temphn = []
for hit in hn:
    for gene in gn:
        hitps=[]
        for item in hitdict:
            if (gene in item) and (hit in item):
                #if %ID < 90%, ignore. O157 is very homogenous, high ID cutoff
                if (int(hitdict[item][4])/int(hitdict[item][3]))<.95:
                    continue
                qlen = int(hitdict[item][2])
                hitps.append((int(hitdict[item][0]),int(hitdict[item][1])))
        hitpss = sorted(hitps, key = lambda x: x[0])
        for i in range(1, len(hitpss)):
            if hitpss[i][0]<hitpss[i-1][1]:
                hitpss[i]=(hitpss[i-1][0],hitpss[i][1])
                hitpss[i-1]=(0,0)
                continue
        if (0,0) in hitpss:
            for i in range(hitpss.count((0,0))):
                hitpss.remove((0,0))

        for i in range(1, len(hitpss)):
            if hitpss[i][0]<hitpss[i-1][1]:
                print('Incomplete trimming')
                continue

        lens = [(i[1]-i[0]+1) for i in hitpss]
        slens = sum(lens)
        if (slens/int(qlen)) >= 0.6:
            ws1.cell(row = hn.index(hit)+2, column = gn.index(gene)+2).fill = fillyes
        else:
            ws1.cell(row = hn.index(hit)+2, column = gn.index(gene)+2).fill = fillno
        ws1.cell(row = hn.index(hit)+2, column = gn.index(gene)+2).value = (slens/int(qlen))
            

wb.save('pSS17_Check.xlsx')

['SS17_RS29050', 'SS17_RS29055', 'SS17_RS29060', 'SS17_RS29065', 'SS17_RS29070', 'SS17_RS29075', 'SS17_RS29080', 'SS17_RS29085', 'SS17_RS29090', 'SS17_RS29095', 'SS17_RS29105', 'SS17_RS29110', 'SS17_RS29115', 'SS17_RS29135', 'SS17_RS29140', 'SS17_RS29145', 'SS17_RS29150', 'SS17_RS29155', 'SS17_RS29160', 'SS17_RS29165', 'SS17_RS29170', 'SS17_RS29175', 'SS17_RS29180', 'SS17_RS29185', 'SS17_RS29190', 'SS17_RS29195', 'SS17_RS29200', 'SS17_RS29205', 'SS17_RS29210', 'SS17_RS29215', 'SS17_RS29220', 'SS17_RS29225', 'SS17_RS29230', 'SS17_RS29235', 'SS17_RS29340', 'SS17_RS29345']
['S07', 'S31', 'S39', 'S41', 'S75', 'S76', 'S77', 'S78', 'S81', 'S87']
